In [ ]:
import Hydrographs as HY
import make_wells as mw
import geopandas as gpd
import basic

In [ ]:
run = f"June{2017}"
ml = basic.load_model(path = f'temp/{run}')
HY.run(run, reload = True, ml = ml)

In [ ]:
asdf

In [ ]:
for year in range(2012, 2022):
    run = f"June{year}"
    ml = basic.load_model(path = f'temp/{run}')
    HY.run(run, reload = True, ml = ml)

In [ ]:
def load_temp(name, datestart=None, numdays=365):
    '''

    :param name:
    :param datestart:
    :param numdays:
    :return:
    '''

    fold = r"T:\arich\Russian_River\MirabelWohler_2022\Waterlevel_Data\MWs_Caissons - AvailableDailyAverages\DailyData\MonitoringWells"

    # need to check if it's a caisson record. if it is, it needs to be loaded differently
    if isinstance(name, str):
        if 'caisson' in name.lower():
            fold = r"T:\arich\Russian_River\MirabelWohler_2022\Waterlevel_Data\MWs_Caissons - AvailableDailyAverages\DailyData\Caissons"
            caisson = True
        else:
            caisson = False

    else:
        caisson = False
        name = 'no filename given'
    
    if caisson:
        path = pathlib.Path(fold).joinpath(name)
    else:
        path = pathlib.Path(fold).joinpath(name.replace('.csv', 'temp.csv'))
    # path = pathlib.Path(fold).joinpath(name.replace(' ', '').replace('-', '_') + '.csv')

    # if end_time is None:
    end_time = pd.to_datetime(datestart) + pd.to_timedelta(numdays, unit='D')

    if path.exists():
        stg = pd.read_csv(path, parse_dates=[0])
        stg = stg.set_index(stg.columns[0])
        stg = stg.resample('1D').mean()
        stg = stg.loc[(stg.loc[:, 'Value'] > 50) & (stg.loc[:, 'Value'] < 120)]

        if datestart is not None:
            stg = stg.loc[datestart:end_time, :]


    else:
        print(f"path does not exist:\n\n{path}\n")

        stg = pd.DataFrame()

    return stg

In [ ]:
import pathlib
import pandas as pd
temp = load_temp('MW93_18.csv')

In [ ]:
import conda_scripts.utils.river_report as rr

In [ ]:
df = rr.load_river_report()

In [ ]:
df.loc[:, 'Caisson 1']

In [ ]:
import conda_scripts.gwplot_fancy as gwp
import pandas as pd

In [ ]:
temp.plot()

In [ ]:
ur1 = r'https://www.kisters.net/sonomacountygroundwater/KiWIS/KiWIS?service=kisters&type=queryServices&request=getStationList&datasource=0&format=html&station_name='
ur2 = r'&returnfields=station_name,station_no,site_name,station_latitude,station_longitude,ca_sta&ca_sta_returnfields=Vert_coord_Loc,TOS_1,BOS_1,TOS_2,BOS_2,TOS_3,BOS_3,TOS_4,BOS_4,TOS_5,BOS_5,TOS_6,BOS_6,TOS_7,BOS_7,TOS_8,BOS_8,'
ur3 = r'Depth_to_Bot_Perf,Depth_to_Top_Perf,Web_GW_Obs_Range,Subbasin_Name,GWREF_DATUM,Web_Station_Name,Well_Depth,Survey_ElevNAD88,Survey_ElevNGVD29,Ref_Elev_NVD29,Survey_ElevNGVD29,Reference_Elevation,Elevation,Height'

url_send = ur1 + 'LRR0225' + ur2 + ur3
x = pd.read_html(url_send, header=0)
x


In [ ]:
nwp.t.get_well_info()

In [ ]:
nwp = gwp.fancy_plot('LRR0225', group=None,
                     filename=None,
                     allinfo=nwp.t.gw_elev.loc[[4],:],
                     do_regress=False)

nwp.do_plot(False, skip_gw_data=False,
            map_buffer_size=2500, seasonal=False,
            plot_dry=False, plot_wet=False,
            maptype='ctx.USGS.USTopo')

ax = nwp.upleft.twinx()
ax.scatter(temp.index, temp.Value,  c = 'r', marker = '.', ls= 'None', label = 'Temp')
ax.set_ylim([30, 75])
ax.legend(loc = 'lower right', bbox_to_anchor = (1,1))
ax.tick_params(axis='y', colors='red')    #setting up X-axis tick color to red
ax.spines['right'].set_color('red')        # setting up Y-axis tick color to red

In [ ]:
temp.plot()

In [ ]:
model_wells = HY.load_wells_mod(ml)

In [ ]:
model_wells

In [ ]:
m = caissons.explore()

In [ ]:
caissons = gpd.read_file('C:\modeling\MirabelWohler\GIS\wells.shp')
caissons = caissons.query('lateral == 0')
caissons.loc[:,'latitude'] = caissons.to_crs(4326).geometry.centroid.y.values
caissons.loc[:,'longitude'] = caissons.to_crs(4326).geometry.centroid.x.values
caissons.loc[:,'northing'] = caissons.to_crs(2226).geometry.centroid.y.values
caissons.loc[:,'easting'] = caissons.to_crs(2226).geometry.centroid.x.values
caissons

In [ ]:
caissons.loc[:,['wellname', 'northing', 'easting']].to_clipboard(sep = ',')

In [ ]:
wells = HY.get_well_locations()

In [ ]:
wells

In [ ]:
import pandas as pd
from basic import isnumber

def load_stg(path):
    stg = pd.read_csv(path)
    stg = stg.rename(columns =  {'DateTime':'Datetime'})
    stg.loc[:,'Datetime'] = pd.to_datetime(stg.loc[:,'Datetime'])
    stg = stg.set_index('Datetime')
    stg = stg.loc[:,['Value']]
    stg.loc[:,'Value'] = stg.loc[:,'Value'].apply(isnumber)
    stg = stg.resample('1D').mean()
    stg = stg.loc[(stg.loc[:, 'Value'] > -50) & (stg.loc[:, 'Value'] < 100)]

    return stg

df = load_stg(r"T:\arich\Russian_River\MirabelWohler_2022\Waterlevel_Data\MWs_Caissons - AvailableDailyAverages\DailyData\Caissons\Caisson1WaterLevel.csv")

df.head()

In [ ]:
df.tail().resample('1D').mean()

In [ ]:
df.head().resample('1D').mean()

In [ ]:
wiski = HY.get_wiski()

In [ ]:
wiski.query("station_name=='LRR0227'")

In [ ]:
w = wiski.loc[wiski.station_no.str.contains('Caisson')]

In [ ]:

wdf = gpd.GeoDataFrame(w, geometry = gpd.points_from_xy(w.station_longitude, w.station_latitude), crs = 4326)

In [ ]:
wdf.explore()